In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))

import torch
import numpy as np
from utils.ply import *
from utils.pc import *

In [3]:
file_name = '/home/chukhran/datasets/completion/shapenet/ShapeNetPointCloud/02691156/10155655850468db78d106ce0a280f87.ply'
point_cloud, _, _  = load_ply(file_name)

In [4]:
point_cloud.shape

(10000, 3)

In [8]:
plot_3d_point_cloud(
    point_cloud, 
    in_u_sphere=True, 
    show=True, 
    title=file_name, 
    backend='qtagg',
    # backend=None
)

Switching backend from module://matplotlib_inline.backend_inline to qtagg...


libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


<Figure size 500x500 with 1 Axes>

In [14]:
import torch
from torch import nn
import torch.nn.functional as F
import torch_geometric 
from torch_geometric.nn.pool import radius_graph
import networkx as nx

In [15]:
graph = torch.from_numpy(point_cloud)
edges = radius_graph(graph, r=0.01)

In [16]:
edges

tensor([[ 665, 6872, 9156,  ..., 9995, 7497, 6074],
        [   0,    0,    0,  ..., 9999, 9999, 9999]])

# Algorithm

1. Count all point in point cloud
2. get edge indices from the point cloud radius graph
3. for each edge (u -> v) get source point coordinates (coord of u) and destination coordinates (coord of v)
4. for this coordinates add a segment

In [12]:
plot_3d_point_cloud(graph.numpy(), edges=edges.numpy()[:, :10000], in_u_sphere=True, show=True, title=file_name, backend='qtagg')

Switching backend from QtAgg to qtagg...


<Figure size 500x500 with 1 Axes>

In [17]:
graph[:5]

tensor([[-0.0477,  0.0515,  0.2680],
        [ 0.1966,  0.0100,  0.0117],
        [-0.0117,  0.0940,  0.2773],
        [ 0.0039,  0.0952,  0.2586],
        [-0.0023,  0.0772,  0.3976]])

In [26]:
conv_nn = nn.Sequential(
    nn.Linear(6, 32),
    nn.LeakyReLU(),
    nn.Linear(32, 64),
)

In [27]:
edge_conv = torch_geometric.nn.EdgeConv(nn=conv_nn, aggr='max')


In [28]:
print(f'Graph props: {graph, graph.shape}')
print(f'Edges props: {edges, edges.shape}')

Graph props: (tensor([[-0.0477,  0.0515,  0.2680],
        [ 0.1966,  0.0100,  0.0117],
        [-0.0117,  0.0940,  0.2773],
        ...,
        [-0.0211,  0.0154,  0.3128],
        [-0.0242,  0.0933,  0.4086],
        [ 0.1508,  0.0110,  0.0273]]), torch.Size([10000, 3]))
Edges props: (tensor([[ 665, 6872, 9156,  ..., 9995, 7497, 6074],
        [   0,    0,    0,  ..., 9999, 9999, 9999]]), torch.Size([2, 126341]))


In [32]:
new_graph = edge_conv(graph, edges)
new_graph, new_graph.shape

(tensor([[-0.0284,  0.1435,  0.0005,  ...,  0.1180, -0.0363, -0.1733],
         [-0.0155,  0.1401,  0.0059,  ...,  0.0751, -0.0076, -0.1881],
         [-0.0209,  0.1423,  0.0091,  ...,  0.1143, -0.0342, -0.1638],
         ...,
         [-0.0308,  0.1500,  0.0029,  ...,  0.1185, -0.0456, -0.1773],
         [-0.0183,  0.1551,  0.0196,  ...,  0.1185, -0.0575, -0.1525],
         [-0.0211,  0.1417,  0.0005,  ...,  0.0796, -0.0063, -0.1884]],
        grad_fn=<ScatterReduceBackward0>),
 torch.Size([10000, 64]))

In [35]:
pooled = torch_geometric.nn.global_max_pool(new_graph, batch=torch.zeros(new_graph.shape[0], dtype=torch.long))
pooled, pooled.shape

(tensor([[0.0000, 0.1587, 0.0305, 0.1214, 0.0000, 0.0798, 0.1614, 0.2050, 0.0876,
          0.0000, 0.0262, 0.0969, 0.1682, 0.2152, 0.0000, 0.0000, 0.0000, 0.0000,
          0.1870, 0.0522, 0.1298, 0.1097, 0.2689, 0.2840, 0.0000, 0.0000, 0.0000,
          0.1388, 0.1748, 0.0000, 0.1512, 0.0000, 0.3260, 0.0000, 0.0000, 0.0000,
          0.1410, 0.2743, 0.0000, 0.1557, 0.0000, 0.1751, 0.0626, 0.1072, 0.0000,
          0.0090, 0.0000, 0.1882, 0.1689, 0.0000, 0.1097, 0.0000, 0.3082, 0.1687,
          0.0000, 0.0000, 0.2098, 0.0000, 0.0000, 0.1294, 0.0643, 0.1255, 0.0492,
          0.0000]], grad_fn=<ScatterReduceBackward0>),
 torch.Size([1, 64]))

In [37]:
farthest = torch_geometric.nn.fps(new_graph, ratio=0.5)
farthest, farthest.shape